In [ ]:
import importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import rcParams
plt.rcParams['xtick.labelsize']  = 18
plt.rcParams['ytick.labelsize']  = 18
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family']      = 'STIXGeneral'
plt.rcParams['axes.labelsize']   = 20
plt.rcParams['axes.titlesize']   = 20
plt.rcParams['font.size']        = 10
plt.rcParams['legend.fontsize']  = 20
plt.rcParams.update({'figure.autolayout': True})
#import scienceplots
#plt.style.use(['science','ieee'])

from tqdm import tqdm
import random
import os
import time
import h5py
from datetime import datetime


import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Conv2DTranspose

# Import auxiliary libraries

import auxLib; importlib.reload(auxLib)

# Main variables

In [ ]:
L = 40

densities = [0.55, 0.6, 0.61, 0.62, 0.63, 0.64, 0.65, 0.7, 0.8, 0.9, 1.0]

densities_str = ['055','06', '061', '062', '063', '064', '065', '07', '08', '09','1']

neurons = 200

seed = 15

callbacks = auxLib.myCallback() # to be able to stop training if accuracy is high enough

In [ ]:
training = ['para', 'ferro', 'neel','stripe']

modelsfolder, resultsfolder = auxLib.folders(directory=None, training=training, percolation=True)


In [ ]:
data = auxLib.SyntheticImagesGen(training=training, L=L)

# Generate synthetic data

train_images, train_labels = data.dataGenerator(10000)
test_images, test_labels = data.dataGenerator(1000)

train_images = train_images.reshape((10000, L * L))
test_images = test_images.reshape((1000, L * L))

# Creating and training models

Now we should use the results of the critical temperatures of each training to decide what number of neurons will we use for percolation problem. We will use 200 neurons.

In [ ]:
np.random.seed(seed)

kernel_regularizer=tf.keras.regularizers.l2(0.01)

dnn = auxLib.DenseNeuralNetworkGen() # Initializing. It generates the model

dnn.build_model((L*L,), num_classes=4, layers=[
{'type': 'dense', 'units': neurons, 'activation': 'relu', 'kernel_regularizer': kernel_regularizer},
{'type': 'dropout', 'rate': 0.2},
{'type': 'dense', 'units': 4, 'activation': 'softmax'} #2 is the number of classes: Paramagnetic and ferromagnetic
])

dnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
dnn.summary()

dnn.fit(train_images, train_labels, 
		epochs=50, batch_size=20,
		validation_data=(test_images, test_labels), 
		callbacks=[callbacks],
		verbose=False)

dnn.save_weights(f'dnn_n{neurons}_DO02', directory=modelsfolder) # name of the model (exclude the 'weights' and the extension)
dnn.save_model(f'dnn_n{neurons}_DO02', directory=modelsfolder)

# Predictions

In [ ]:
# Simulated images

loader_init = auxLib.loader_and_saver(os.getcwd())


In [ ]:

for densitiy_index, density in enumerate(densities):
    
    sim_images, temperature = loader_init.simulatedImages(densitiy_index)

    model_path = os.path.join(modelsfolder, f'dnn_n{neurons}_DO02.h5')

    dnn = keras.models.load_model(model_path)

    prediction = auxLib.predictor(L, dnn, sim_images, neurons,
								training=training,
                                directory=resultsfolder, 
                                reshape = True, 
                                verbose=False, 
                                save_prediction=True)


# Calculating critical temperature

In [ ]:
critical_temperatures = []

for neuron in neurons:
    prediction = pd.read_csv(resultsfolder+f'/predictions_{neuron}.csv', header=0).to_numpy(float)
    temperature, para, neel, stripe = prediction.T

    print(f'Prediction for {neuron} neurons in hidden layer')

    plt.figure(figsize=(7, 7))
    plt.plot(temperature, para, '^-', label='Paramagnetic', markerfacecolor='none', markersize=7)
    plt.plot(temperature, ferro, 'o-', label='Ferromagnetic', markerfacecolor='none', markersize=7)
    
    if neuron<8:
        plt.xlim((0.5, 3))
    else:
        plt.xlim((1.5, 3))
    plt.xlabel('Temperature')
    plt.ylabel('Probability')
    plt.legend()

    if max(ferro)<0.5:
        tc = 0
    else:
        index_half = np.where(ferro > 0.5)[0][-1]
        if index_half == 250:
            index_half = 249
        # Interpolation:
        tc = temperature[index_half] + (ferro[index_half]-0.5)*(temperature[index_half+1] - temperature[index_half]) / (ferro[index_half] - ferro[index_half+1])
        plt.axvline(x=tc,  color = 'red', linestyle = '--', alpha = 0.5)

    critical_temperatures.append(tc)
    print(f'T_c = {tc}')
    plt.savefig(os.path.join(resultsfolder, f'{neuron}neurons.pdf'))
    plt.show()

In [ ]:
temp = pd.DataFrame((np.concatenate((np.array(neurons).reshape(len(neurons),1), 
                                    np.array(critical_temperatures).reshape(len(neurons),1)),axis=1)), 
                                    columns=['Neurons','Temperature'])
temp.to_csv(os.path.join(resultsfolder, 'critical_temperatures.csv'), index=False)

In [ ]:
temp = pd.read_csv(os.path.join(resultsfolder, f'critical_temperatures.csv'))

plt.figure(figsize=(7, 7))
plt.scatter(temp['Neurons'][1:],temp['Temperature'][1:], marker='x', color='black', label='Critical temperature')
plt.axhline(y=2.269, color = 'red', linestyle = '--', label = 'Theoretical critical temperature')
plt.xlabel('Neurons')
plt.ylabel('Temperature')

#plt.text(800, 2.25, rf'Tc ={2.269}', color='black', 
#        verticalalignment='top', horizontalalignment='left', fontsize=20)

plt.ylim((2, 3))
plt.legend(loc='upper right')
plt.savefig(os.path.join(os.getcwd(), resultsfolder, 'critical_temperatures.pdf'))
plt.show()

# Calculating critical temperature using Half-height method

In [ ]:
training = ['para','ferro']

modelsfolder, resultsfolder = auxLib.folders(directory=None, training=training)
saving_folder = os.path.join(os.getcwd(), 'DNN', 'Para_Neel_Stripe', 'predictions', datetime.now().strftime('%Y-%m-%d'))
os.makedirs(saving_folder, exist_ok = True)

critical_temperatures = []

for neuron in neurons:
	print(f'Prediction for {neuron} neurons in hidden layer')
	tc, plot = auxLib.half_height(resultsfolder, name=f'predictions_{neuron}', prediction=None, graph=True)
	plot.savefig(saving_folder+f'/critical_temperature_{neuron}.pdf') #saving the figure
	plot.show()
	critical_temperatures.append(tc)

temp = pd.DataFrame((np.concatenate((np.array(neurons).reshape(len(neurons),1), 
                                    np.array(critical_temperatures).reshape(len(neurons),1)),axis=1)), 
                                    columns=['Neurons','Temperature'])
temp.to_csv(os.path.join(saving_folder, 'critical_temperatures(half).csv'), index=False)

In [ ]:
data_df = pd.read_csv(os.path.join(saving_folder,f'critical_temperatures(half).csv'), header=0)

plt.figure(figsize=(7, 7))

plt.scatter(data_df['Neurons'], data_df['Temperature'], marker='x', color='black', label='Critical temperature') # excluding the first neuron
plt.axhline(y=2.269, color = 'red', linestyle = '--', label = 'Theoretical critical temperature')

plt.xlabel('Number of neurons in hidden layer')
plt.ylabel('Critical temperature')

plt.text(250, 2.269, rf'Tc ={2.269}', color='black', 
        verticalalignment='bottom', horizontalalignment='right', fontsize=20)

plt.ylim((2, 3))
plt.legend()
plt.savefig(os.path.join(saving_folder, 'critical_temperature(half).pdf'))
plt.show()